<a href="https://colab.research.google.com/github/shyamgv/ImageCaptions/blob/main/Finetuning_imageds_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets jiwer evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip uninstall -y pyarrow
!pip install pyarrow==14.0.1

Found existing installation: pyarrow 16.1.0
Uninstalling pyarrow-16.1.0:
  Successfully uninstalled pyarrow-16.1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 24.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.6.1 requires pyarrow<16.2.0a0,>=16.1.0, but you have pyarrow 14.0.1 which is incompatible.
datasets 3.0.1 requires pyarrow>=15.0.0, but you have pyarrow 14.0.1 which is incompatible.


In [4]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 76.1 MB/s eta 0:00:00


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [25]:
## Creating own image caption dataset
import json

captions = [{"file_name": "IMG_3337.jpg", "text": "Grandfather with Sai at market basket in a chair"},
            {"file_name": "IMG_3338.jpg", "text": "Grandfather with Sai at market basket in a chair"},
            {"file_name": "IMG_0721.jpg", "text": "Shyam and Santoshi doing ritual with children with others around"},
            {"file_name": "IMG_3463.jpg", "text": "Shyam's I-797A"},
            {"file_name": "IMG_3461.jpg", "text": "Shyam's I-797A"},
            {"file_name": "IMG_3328.jpg", "text": "Shyam's India Driving License"},
            {"file_name": "IMG_0749.jpg", "text": "Sai sitting on a Table"},
            {"file_name": "IMG_0724.jpg", "text": "Rama in Krishna's costume"},
            {"file_name": "IMG_0714.jpg", "text": "Rama with his sister"},
            {"file_name": "IMG_0639.jpg", "text": "Backside of drivers license"},
            {"file_name": "IMG_0638.jpg", "text": "Frontside of US drivers license"},
            {"file_name": "IMG_0108.jpg", "text": "Backside of health insurance card"},
            {"file_name": "IMG_0001.jpg", "text": "School calendar"},
            {"file_name": "IMG_0107.jpg", "text": "Frontside of health insurance card"},
            {"file_name": "IMG_0629.jpg", "text": "Santoshi with Rama at dawn in a park"},
            {"file_name": "IMG_0703.jpg", "text": "Sai and Rama together at home"},
            {"file_name": "IMG_0498.jpg", "text": "Sai and Rama with his aunty and uncle at home"},
            {"file_name": "IMG_0497.jpg", "text": "Sai and Rama with his aunty and uncle at home"},
            {"file_name": "IMG_0744.jpg", "text": "Sai and Rama in Krishna's costume"},
            {"file_name": "IMG_0745.jpg", "text": "Sai and Rama in Krishna's costume with Santoshi"},
            {"file_name": "IMG_0751.jpg", "text": "Sai and Rama having fun outside"},
            {"file_name": "IMG_0722.jpg", "text": "Shyam performing a ritual with others"},
            {"file_name": "IMG_0753.jpg", "text": "Rama in a fancy chair outside"},
            {"file_name": "IMG_0616.jpg", "text": "Santoshi and Rama playing in a playground"},
            {"file_name": "IMG_0677.jpg", "text": "Santoshi feeding Rama at a public restaurant"},
            {"file_name": "IMG_0795.jpg", "text": "Sai and Rama enjoying brother bonding at home"},
            {"file_name": "IMG_7136.jpg", "text": "Shyam and Santoshi family picture"},
            {"file_name": "IMG_0766.jpg", "text": "Rama playing at home"},
            {"file_name": "IMG_0758.jpg", "text": "Santoshi and Rama enjoying their time outside"},
            {"file_name": "IMG_0783.jpg", "text": "Shyam and Rama enjoying father son time at home"},
            {"file_name": "IMG_0788.jpg", "text": "Rama at home behind TV"},
            {"file_name": "IMG_0816.jpg", "text": "Rama at home opening kitchen cabinets"},
            {"file_name": "IMG_0779.jpg", "text": "Sai happy at home"},
            {"file_name": "IMG_0780.jpg", "text": "Sai enjoying in the playground"},
            {"file_name": "IMG_0782.jpg", "text": "Sai enjoying in the playground"},
            {"file_name": "IMG_0830.jpg", "text": "Sai sitting on sofa with a vegetable"},
            {"file_name": "IMG_0825.jpg", "text": "Sai enjoying in a swing"},
            {"file_name": "IMG_0826.jpg", "text": "Santoshi passport size photo"},
            ]
root = "/content/drive/MyDrive/Phone_Images/"

with open(root + "metadata.jsonl", 'w') as f:
  for item in captions:
    f.write(json.dumps(item) + "\n")

In [ ]:
## Creating own image caption dataset
import json

captions = [
            {"file_name": "IMG_0749.jpg", "text": "Sai sitting on a Table"},
            {"file_name": "IMG_0724.jpg", "text": "Rama in Krishna's costume"},
            {"file_name": "IMG_0638.jpg", "text": "Shyam's US drivers license"},
            {"file_name": "IMG_0703.jpg", "text": "Sai and Rama together at home"},
            {"file_name": "IMG_3337.jpg", "text": "Grandfather with Sai at market basket in a chair"},
            {"file_name": "IMG_0825.jpg", "text": "Sai on a swing"},
            {"file_name": "IMG_0760.jpg", "text": "Sai"},
            {"file_name": "IMG_0664.jpg", "text": "Sai and Rama in temple"},
            {"file_name": "IMG_0661.jpg", "text": "Sai with Baba"},
            {"file_name": "IMG_0674.jpg", "text": "Sai in temple canteen"},
            {"file_name": "IMG_0753.jpg", "text": "Rama in a fancy chair outside"},
            {"file_name": "IMG_0766.jpg", "text": "Rama playing at home"},
            {"file_name": "IMG_0768.jpg", "text": "Rama playing at home"},
            {"file_name": "IMG_0767.jpg", "text": "Rama happy at home"},
            {"file_name": "IMG_0841.jpg", "text": "Santoshi at home"},
            {"file_name": "IMG_0839.jpg", "text": "Rama eating"},
            {"file_name": "IMG_0838.jpg", "text": "Shyam"},
            {"file_name": "IMG_0842.jpg", "text": "Santoshi at home"},
            {"file_name": "IMG_0765.jpg", "text": "Rama at home"},
            {"file_name": "IMG_001.jpg", "text": "Santoshi at home"},
            ]
root = "/content/drive/MyDrive/Training_Images/"

with open(root + "metadata.jsonl", 'w') as f:
  for item in captions:
    f.write(json.dumps(item) + "\n")

In [26]:
from datasets import load_dataset


dataset = load_dataset("imagefolder", data_dir="/content/drive/MyDrive/Phone_Images", split="train").train_test_split(test_size=0.1)
#test_ds = load_dataset("imagefolder", data_dir="/content/drive/MyDrive/Phone_Images", split="test[20%:]")
train_ds = dataset["train"]
test_ds = dataset["test"]
#test_ds

Resolving data files:   0%|          | 0/38 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
#from transformers import AutoProcessor
from transformers import BlipProcessor

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large",clean_up_tokenization_spaces = True)
#processor = AutoProcessor.from_pretrained("microsoft/git-base")

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [28]:
def transforms(example_batch):
    images = [x for x in example_batch["image"]]
    captions = [x for x in example_batch["text"]]
    inputs = processor(images=images, text=captions, padding="max_length")
    inputs.update({"labels": inputs["input_ids"]})
    return inputs


train_ds.set_transform(transforms)
test_ds.set_transform(transforms)

In [2]:
#from transformers import AutoModelForCausalLM
from transformers import BlipForConditionalGeneration
#model = AutoModelForCausalLM.from_pretrained("microsoft/git-base")
checkpoint = "shyamgv/blip-image-captioning-large-shyam"
model = BlipForConditionalGeneration.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/839 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [30]:
from evaluate import load
import torch

wer = load("wer")

def compute_metrics(eval_pred):
  logits_touple = eval_pred[0]
  logits = logits_touple[0]
  print(f"logits are: {logits}")
  labels = eval_pred[1]
  predicted = logits.argmax(-1)
  decoded_labels = processor.batch_decode(labels, skip_special_tokens = True)
  decoded_predictions = processor.batch_decode(predicted, skip_special_tokens = True)
  wer_score = wer.compute(predictions=decoded_predictions, references=decoded_labels)
  return {"wer_score": wer_score}

In [38]:
from transformers import TrainingArguments, Trainer
import torch

model_name = checkpoint.split("/")[1]

training_args = TrainingArguments(
    output_dir=f"{model_name}",
    #optim=torch.optim.AdamW,
    learning_rate=5e-5,
    num_train_epochs=15,
    fp16=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,
    save_total_limit=3,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    logging_steps=50,
    remove_unused_columns=False,
    push_to_hub=True,
    label_names=["labels"],
    load_best_model_at_end=True,

)

In [39]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [33]:
trainer.train()

Step,Training Loss,Validation Loss,Wer Score
50,0.010400,0.004082,0.111111
100,0.002500,0.004877,0.111111
150,0.000300,0.005648,0.166667
200,0.000200,0.004469,0.111111
250,0.000100,0.004422,0.111111


logits are: [[[ 1.8271484  -2.7207031  -2.7207031  ... -2.7207031  -2.7109375
   -2.7226562 ]
  [-0.31445312 -2.5957031  -2.5957031  ... -2.5957031  -2.4902344
   -2.5976562 ]
  [ 1.4853516  -3.0019531  -3.0019531  ... -3.0019531  -3.0683594
   -3.0039062 ]
  ...
  [12.75       -4.3554688  -4.3554688  ... -4.3554688  -4.4726562
   -4.3632812 ]
  [12.7578125  -4.3554688  -4.3554688  ... -4.3554688  -4.4726562
   -4.359375  ]
  [12.75       -4.359375   -4.359375   ... -4.359375   -4.4765625
   -4.3671875 ]]

 [[ 1.1748047  -2.6269531  -2.6269531  ... -2.6269531  -2.59375
   -2.6269531 ]
  [ 0.6689453  -1.7871094  -1.7871094  ... -1.7871094  -1.7275391
   -1.7880859 ]
  [ 3.5898438  -1.8164062  -1.8173828  ... -1.8164062  -1.8251953
   -1.8183594 ]
  ...
  [12.765625   -4.34375    -4.34375    ... -4.34375    -4.4609375
   -4.3476562 ]
  [12.765625   -4.3398438  -4.3398438  ... -4.3398438  -4.4570312
   -4.34375   ]
  [12.765625   -4.34375    -4.34375    ... -4.34375    -4.4609375
   -4.34

There were missing keys in the checkpoint model loaded: ['text_decoder.cls.predictions.decoder.bias'].


TrainOutput(global_step=255, training_loss=0.0026675020472383968, metrics={'train_runtime': 1069.2413, 'train_samples_per_second': 0.463, 'train_steps_per_second': 0.238, 'total_flos': 5.85834537983017e+17, 'train_loss': 0.0026675020472383968, 'epoch': 15.0})

In [40]:
trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shyamgv/blip-image-captioning-large-shyam/commit/b05a6f4718bfb697932361fe3cfc0c28d372610c', commit_message='End of training', commit_description='', oid='b05a6f4718bfb697932361fe3cfc0c28d372610c', pr_url=None, pr_revision=None, pr_num=None)

In [16]:
# load image
from PIL import Image

image = Image.open("/content/drive/MyDrive/Test_Images/IMG_0787.jpg")
#image

In [36]:
#counter = 0
#for ds in dataset["test"]:
#  text = ds["text"]
#  print(text, "  ", counter)
#  counter += 1

In [17]:
# prepare image for the model
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
inputs = processor(images=image, return_tensors="pt").to(device)
pixel_values = inputs.pixel_values

generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_caption)

shyam and santoshi father son time at home
